In [1]:
import pandas as pd
import numpy as np

In [2]:
sales = pd.read_csv("NoteBook/processed_sales_data.csv")          # columns: id, item_id, dept_id, cat_id, store_id, state_id, date, units_sold
calendar = pd.read_csv("Data/calendar.csv")    # columns: date, wm_yr_wk, event_name_1, event_type_1, event_name_2, event_type_2
prices = pd.read_csv("Data/sell_prices.csv")   # columns: store_id, item_id, wm_yr_wk, sell_price

In [4]:
# Chuyển sang parquet để tiết kiệm dung lượng lưu trữ và tăng tốc độ đọc dữ liệu
sales.to_parquet("Data/Parquet/processed_sales_data.parquet", index=False)
calendar.to_parquet("Data/Parquet/calendar.parquet", index=False)
prices.to_parquet("Data/Parquet/sell_prices.parquet", index=False)

In [7]:
sales = sales.merge(calendar[["date", "wm_yr_wk"]], on="date", how="left")

# Tổng hợp theo tuần
sales_weekly = (
    sales.groupby(
        ["item_id", "dept_id", "cat_id", "store_id", "state_id", "wm_yr_wk"],
        as_index=False
    ).agg(
        weekly_units_sold=("units_sold", "sum")
    )
)

sales_weekly.to_csv("sales_weekly.csv", index=False)

In [8]:

calendar["date"] = pd.to_datetime(calendar["date"])

def select_main_event(subdf):
    # Nếu không có event thì bỏ qua
    event_days = subdf[subdf["event_name_1"].notna() | subdf["event_name_2"].notna()]
    if event_days.empty:
        return pd.Series({"main_event_name": None, "main_event_type": None})

    # Chọn event đầu tiên trong tuần (hoặc có thể chọn ngày event_type quan trọng hơn)
    first_event = event_days.iloc[0]
    event_name = first_event["event_name_1"] or first_event["event_name_2"]
    event_type = first_event["event_type_1"] or first_event["event_type_2"]

    return pd.Series({"main_event_name": event_name, "main_event_type": event_type})

calendar_weekly = (
    calendar.groupby("wm_yr_wk")
            .apply(select_main_event)
            .reset_index()
)

calendar_weekly.to_csv("calendar_weekly.csv", index=False)


┌───────────────────────────────── ⚠️ Warning ─────────────────────────────────┐
│ FutureWarning in                                                            │
│ C:\Users\Admin\AppData\Local\Temp\ipykernel_11104\3953763295.py:18          │
│ DataFrameGroupBy.apply operated on the grouping columns. This behavior is   │
│ deprecated, and in a future version of pandas the grouping columns will be  │
│ excluded from the operation. Either pass `include_groups=False` to exclude  │
│ the groupings or explicitly select the grouping columns after groupby to    │
│ silence this warning.                                                       │
└─────────────────────────────────────────────────────────────────────────────┘


In [9]:


prices_weekly = (
    prices.groupby(["store_id", "item_id", "wm_yr_wk"], as_index=False)
          .agg(
              avg_sell_price=("sell_price", "mean")
          )
)

prices_weekly.to_csv("prices_weekly.csv", index=False)
